# Bab 12: Custom Models and Training with TensorFlow (Model Kustom dan Pelatihan dengan TensorFlow)

### 1. Pendahuluan

Bab 11 telah membahas berbagai teknik untuk melatih *Deep Neural Networks* menggunakan Keras API yang *high-level*. Bab 12 ini akan menyelam lebih dalam ke TensorFlow, mengeksplorasi cara membangun model kustom, mendefinisikan lapisan kustom, fungsi *loss* kustom, metrik kustom, dan bahkan *training loop* kustom. Ini diperlukan ketika Keras Sequential atau Functional API tidak cukup fleksibel untuk arsitektur atau alur kerja yang sangat spesifik.

Bab ini berfokus pada fleksibilitas dan kontrol yang ditawarkan oleh TensorFlow yang lebih *low-level* (tetapi masih dalam konteks TensorFlow 2.x dengan *eager execution*).

### 2. TensorFlow Low-Level API (API TensorFlow Tingkat Rendah)

Meskipun Keras sangat berguna, memahami API TensorFlow tingkat rendah memberikan kontrol yang lebih besar dan pemahaman yang lebih dalam tentang apa yang terjadi di balik layar.

#### a. Tensors and Operations (Tensor dan Operasi)
* **Tensor:** Representasi dasar data di TensorFlow. Mirip dengan array NumPy, tetapi dapat berjalan di GPU atau TPU.
* **Operasi (Operations):** Fungsi yang memanipulasi *tensor*. TensorFlow menyediakan berbagai operasi matematika, aljabar linier, dan lainnya.
* TensorFlow secara otomatis membangun grafik komputasi (grafik operasi) di belakang layar, bahkan dengan *eager execution*. Grafik ini dioptimalkan untuk kinerja dan dapat dijalankan di berbagai perangkat.

#### b. Graphs and Sessions (Grafik dan Sesi)
* Pada TensorFlow 1.x, pengguna harus secara eksplisit membangun grafik komputasi dan menjalankannya dalam sebuah sesi.
* Pada TensorFlow 2.x (dengan *eager execution* aktif secara *default*), eksekusi bersifat imperatif (operasi segera dieksekusi, mirip Python biasa). Namun, TensorFlow masih membangun grafik secara internal untuk optimisasi kinerja (misalnya, dengan menggunakan `@tf.function`).
* **`@tf.function`:** Dekorator Python yang mengkompilasi fungsi Python menjadi grafik TensorFlow yang sangat efisien. Ini sangat penting untuk meningkatkan kinerja kode yang ditulis dalam *eager execution*.

### 3. Custom Loss Functions (Fungsi Loss Kustom)

Ketika fungsi *loss* standar (misalnya, `mse`, `categorical_crossentropy`) tidak memadai untuk masalah Anda, Anda dapat membuat fungsi *loss* kustom.

* Fungsi *loss* kustom harus menerima `y_true` (label sebenarnya) dan `y_pred` (prediksi model) sebagai input dan mengembalikan satu *scalar* yang mewakili nilai *loss*.
* Contoh fungsi *loss* khusus untuk regresi yang memberikan bobot lebih pada *error* besar (Huber loss) ditunjukkan.

### 4. Custom Activation Functions, Initializers, Regularizers, and Constraints (Fungsi Aktivasi, Initializer, Regularizer, dan Constraint Kustom)

Anda dapat membuat fungsi atau objek kustom untuk aspek-aspek lain dari model Anda:

* **Fungsi Aktivasi Kustom:** Fungsi Python yang menerima *tensor* sebagai input dan mengembalikan *tensor* yang diaktifkan.
* **Initializer Kustom:** Fungsi atau kelas yang menginisialisasi bobot lapisan.
* **Regularizer Kustom:** Fungsi atau kelas yang menambahkan penalti ke *loss*.
* **Constraint Kustom:** Fungsi atau kelas yang menerapkan kendala pada bobot lapisan (misalnya, `tf.keras.constraints.max_norm`).

Anda dapat menyertakan fungsi kustom ini dalam model Keras Anda dengan melewatkannya sebagai argumen ke lapisan yang relevan.

### 5. Custom Models (Model Kustom)

Keras Subclassing API (yang diperkenalkan di Bab 10) adalah cara yang lebih fleksibel untuk membangun model yang sepenuhnya kustom.

* Anda mewarisi dari `tf.keras.Model`.
* Inisialisasi lapisan Anda di metode `__init__()`.
* Definisikan *forward pass* di metode `call()`.
* Ini memungkinkan model dengan *loops*, logika kondisional, atau berbagai arsitektur dinamis.
* Contoh model dengan *skip connection* atau *residual blocks* dapat diimplementasikan dengan mudah menggunakan pendekatan ini.

### 6. Custom Layers (Lapisan Kustom)

Ketika Anda membutuhkan lapisan yang melakukan sesuatu yang tidak standar (misalnya, lapisan yang tidak memiliki bobot, atau memiliki beberapa input/output yang aneh), Anda dapat membuat lapisan kustom.

* Anda mewarisi dari `tf.keras.layers.Layer`.
* Metode `__init__()`: Buat sub-lapisan dan variabel (weights) yang akan digunakan oleh lapisan ini. Gunakan `self.add_weight()`.
* Metode `build(input_shape)`: Ini dipanggil pertama kali lapisan digunakan. Di sinilah Anda dapat membuat variabel-variabel yang bergantung pada bentuk input.
* Metode `call(inputs)`: Definisikan *forward pass* lapisan.
* Metode `compute_output_shape(input_shape)`: Definisikan bentuk output lapisan (opsional, TensorFlow dapat menyimpulkan ini secara otomatis di banyak kasus).

### 7. Custom Training Loops (Loop Pelatihan Kustom)

Meskipun `model.fit()` sangat nyaman, terkadang Anda membutuhkan kontrol penuh atas proses pelatihan. Ini mungkin diperlukan untuk algoritma pelatihan yang sangat spesifik (misalnya, *GANs*, *Reinforcement Learning*), atau jika Anda ingin mengimplementasikan teknik pelatihan tingkat rendah yang tidak didukung langsung oleh Keras.

Langkah-langkah umum dalam *training loop* kustom:
1.  **Mendefinisikan *Optimizer* dan *Loss Function*.**
2.  **Mengiterasi *Epochs*:** Loop untuk setiap *epoch*.
3.  **Mengiterasi *Batches*:** Loop untuk setiap *mini-batch* dari data.
4.  **Menghitung Gradien (Forward Pass + Backward Pass):**
    * Menggunakan `tf.GradientTape()` untuk merekam operasi dan menghitung gradien.
    * `with tf.GradientTape() as tape:`
    * `loss = loss_fn(y_true, y_pred)`
    * `gradients = tape.gradient(loss, model.trainable_variables)`
5.  **Memperbarui Parameter:**
    * `optimizer.apply_gradients(zip(gradients, model.trainable_variables))`
6.  **Memperbarui Metrik:** Menggunakan `tf.keras.metrics` untuk melacak kinerja.
7.  **Menampilkan Kemajuan:** Mencetak metrik di setiap *epoch*.

### 8. TensorFlow Functions (Fungsi TensorFlow)

Menggunakan `@tf.function` adalah kunci untuk mendapatkan kinerja yang baik dari kode TensorFlow yang ditulis dalam *eager execution*. Ini mengkompilasi fungsi Python menjadi grafik TensorFlow yang dapat dioptimalkan dan dijalankan dengan sangat efisien.

* Ketika fungsi dengan `@tf.function` pertama kali dipanggil, TensorFlow akan menelusuri kodenya untuk membuat grafik.
* Panggilan selanjutnya akan langsung menjalankan grafik yang sudah dikompilasi, yang jauh lebih cepat daripada eksekusi *eager* biasa.
* **Aturan untuk `@tf.function`:** Hindari penggunaan *Python side effects* (misalnya, mencetak, memodifikasi daftar Python) di dalam fungsi yang dihiasi, karena efek samping ini hanya terjadi pada jejak pertama.

### 9. Kesimpulan

Bab 12 adalah panduan esensial untuk para ahli dan mereka yang membutuhkan fleksibilitas maksimum di TensorFlow. Ini menunjukkan bagaimana membuat fungsi *loss*, aktivasi, *initializer*, *regularizer*, dan *constraint* kustom. Lebih penting lagi, ia mendemonstrasikan bagaimana membangun model dan lapisan kustom menggunakan Subclassing API, serta mengimplementasikan *training loop* kustom yang memberikan kontrol penuh atas proses pembelajaran. Penggunaan `@tf.function` adalah kunci untuk mengkompilasi dan mengoptimalkan kode yang kompleks menjadi grafik TensorFlow yang berkinerja tinggi.

## 1. Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

### Loading Fashion MNIST (as used in previous chapters)

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## 2. TensorFlow Low-Level API

### Tensors and Operations

In [3]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # a matrix
tf.constant(42) # a scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t[:, 1:]
t[..., 1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

In [5]:
t + 10
tf.square(t)
t @ tf.transpose(t) # Matrix multiplication

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### Graphs and Sessions (TensorFlow 2.x with @tf.function)

In [6]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [7]:
@tf.function
def compiled_huber_fn(y_true, y_pred):
    return huber_fn(y_true, y_pred)

## 3. Custom Loss Functions

In [8]:
def create_huber_loss(threshold=1.0):
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - 0.5 * threshold**2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_loss

In [9]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# Use the custom loss function
model.compile(loss=create_huber_loss(threshold=2.0), optimizer="nadam")
# Dummy data for regression example (Fashion MNIST is classification)
X_dummy = tf.constant(np.random.rand(100, 28, 28).astype(np.float32))
y_dummy = tf.constant(np.random.rand(100, 1).astype(np.float32))
history = model.fit(X_dummy, y_dummy, epochs=5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 0.7612
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.5685
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.4371 
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.3132 
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2080 


## 4. Custom Activation Functions, Initializers, Regularizers, and Constraints

### Custom Activation Function

In [10]:
def my_softplus(z): # custom activation function
    return tf.math.log(tf.exp(z) + 1.0)

# Example:
keras.layers.Dense(30, activation=my_softplus, input_shape=X_train.shape[1:])

<Dense name=dense_2, built=False>

### Custom Glorot Initializer

In [11]:
# Default in Keras Dense layer is Glorot uniform.
# You can customize it like this:
# keras.layers.Dense(30, kernel_initializer="glorot_normal", activation="relu")

# Custom Initializer example:
def my_glorot_initializer(shape, dtype=tf.float32):
    fan_in, fan_out = shape
    limit = tf.math.sqrt(2.0 / (fan_in + fan_out))
    return tf.random.uniform(shape, -limit, limit, dtype)

# Example usage:
keras.layers.Dense(30, kernel_initializer=my_glorot_initializer, activation="relu")

<Dense name=dense_3, built=False>

### Custom Regularizer

In [12]:
def l1_regularizer(weight_matrix):
    return tf.reduce_sum(tf.abs(0.01 * weight_matrix))

# Example:
keras.layers.Dense(30, activation="relu", kernel_regularizer=l1_regularizer)

<Dense name=dense_4, built=False>

### Custom Constraint

In [13]:
def custom_max_norm(weights):
    return tf.clip_by_norm(weights, clip_norm=3.0, axes=0)

# Example:
keras.layers.Dense(30, activation="relu", kernel_constraint=custom_max_norm)

<Dense name=dense_5, built=False>

## 5. Custom Models (Subclassing API)

In [14]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1) # Example for multiple outputs

    def call(self, inputs):
        input_A, input_B = inputs # Assuming two inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output # Return tuple for multiple outputs

# Example usage (requires dummy inputs)
X_train_A = np.random.rand(100, 5) # dummy wide input
X_train_B = np.random.rand(100, 6) # dummy deep input
y_train_main = np.random.rand(100, 1) # dummy main output
y_train_aux = np.random.rand(100, 1) # dummy auxiliary output

model_custom = WideAndDeepModel()
model_custom.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")
history = model_custom.fit((X_train_A, X_train_B), (y_train_main, y_train_aux), epochs=10)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3662 - mse_loss: 0.3903 
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2823 - mse_loss: 0.3666 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2080 - mse_loss: 0.3491
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1935 - mse_loss: 0.3659
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1561 - mse_loss: 0.3469
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1458 - mse_loss: 0.3565
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1285 - mse_loss: 0.3096
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1329 - mse_loss: 0.3070
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1255 - mse_loss: 0.3127
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1242 - mse_loss: 0.3090


## 6. Custom Layers

In [15]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # Residual connection

In [16]:
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

# Example usage (needs data and compilation)
model_res = ResidualRegressor(1)
model_res.compile(loss="mse", optimizer="adam")
history = model_res.fit(np.random.rand(100, 28, 28), np.random.rand(100, 1), epochs=5)

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.6145
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4329
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2678
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2532
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1772


## 7. Custom Training Loops

In [17]:
# Create a simple model for custom training loop
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[1])])

# Create dummy data
X_train_custom = tf.constant(np.arange(100).reshape(100, 1), dtype=tf.float32)
y_train_custom = tf.constant(np.arange(100).reshape(100, 1) * 2 + 1, dtype=tf.float32) # y = 2x + 1

# Define loss and optimizer
loss_fn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.SGD(learning_rate=0.01)

# Training loop
n_epochs = 5
batch_size = 32
n_batches = int(np.ceil(len(X_train_custom) / batch_size))

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    for i in range(n_batches):
        X_batch = X_train_custom[i * batch_size:(i + 1) * batch_size]
        y_batch = y_train_custom[i * batch_size:(i + 1) * batch_size]
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            loss = loss_fn(y_batch, y_pred)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f"  Loss: {loss.numpy()}") # Print final batch loss for epoch

Epoch 1/5
  Loss: 110105051791360.0
Epoch 2/5
  Loss: 4.075533025238498e+27
Epoch 3/5
  Loss: inf
Epoch 4/5
  Loss: inf
Epoch 5/5
  Loss: inf


### Custom Metrics

In [21]:
# Assuming create_huber_loss function is defined earlier in the notebook
# as provided in the initial context.

class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, name=None, dtype=None, **kwargs):
        super().__init__(name=name, dtype=dtype, **kwargs)
        self.threshold = threshold
        # Reusing the custom loss function defined earlier
        self.huber_fn = create_huber_loss(threshold)
        # Explicitly specify shape=() for scalar variables
        self.total = self.add_weight(name="total", shape=(), initializer="zeros", dtype=tf.float32)
        self.count = self.add_weight(name="count", shape=(), initializer="zeros", dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Ensure y_true and y_pred have compatible shapes and types
        # Reshape y_pred to match y_true if necessary and possible
        if y_true.shape.rank != y_pred.shape.rank:
            # Attempt to reshape y_pred to match y_true's rank if y_true is (batch_size, 1)
            if y_true.shape.rank == 2 and y_pred.shape.rank == 1:
                 y_pred = tf.expand_dims(y_pred, axis=-1)
            # Handle other rank mismatches if necessary, or raise an error
            else:
                 # This case is less likely with a simple sequential model
                 # but good practice to acknowledge.
                 tf.print("Shape rank mismatch in HuberMetric.update_state:",
                          "y_true shape:", tf.shape(y_true),
                          "y_pred shape:", tf.shape(y_pred))
                 # You might want to raise an error here if unexpected
                 # raise ValueError("Shape rank mismatch between y_true and y_pred")

        # Ensure y_true and y_pred have the same float type for calculations
        y_pred = tf.cast(y_pred, self.dtype)
        y_true = tf.cast(y_true, self.dtype)

        # Calculate Huber loss for the current batch
        # Ensure shapes are compatible for subtraction after casting/reshaping
        if y_true.shape != y_pred.shape:
             tf.print("Shape mismatch after casting/reshaping in HuberMetric.update_state:",
                      "y_true shape:", tf.shape(y_true),
                      "y_pred shape:", tf.shape(y_pred))
             # You might want to raise an error here if unexpected
             # raise ValueError("Shape mismatch between y_true and y_pred after processing")


        metric_values = self.huber_fn(y_true, y_pred)

        # Sum the loss values and add to the total
        self.total.assign_add(tf.reduce_sum(tf.cast(metric_values, self.dtype)))
        # Count the number of samples in the batch and add to the count
        self.count.assign_add(tf.cast(tf.size(y_true), self.dtype))

    def result(self):
        # Compute the average Huber loss, ensuring division is float
        # Add epsilon for stability in case count is zero
        return self.total / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        # Reset the state variables for a new epoch or evaluation
        self.total.assign(0.0)
        self.count.assign(0.0)

# Define a simple sequential model suitable for regression (single output neuron)
# Add a Flatten layer to handle image input
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=X_train.shape[1:]), # Explicitly flatten the 28x28 images
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1) # Single output for regression
])

# Compile the model, specifying the loss, optimizer, and the custom metric
model.compile(loss="mse", optimizer="nadam", metrics=[HuberMetric(threshold=2.0, name="huber_metric")])

# Create dummy data suitable for a regression problem
# Input shape should match the Flatten layer's expected input (28, 28)
X_dummy = tf.constant(np.random.rand(100, 28, 28).astype(np.float32))
# Target data shape should match the model's output shape (1,)
y_dummy = tf.constant(np.random.rand(100, 1).astype(np.float32))

# Train the model using the dummy data.
history = model.fit(X_dummy, y_dummy, epochs=5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - huber_metric: 0.2713 - loss: 0.5426
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - huber_metric: 0.1647 - loss: 0.3295
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - huber_metric: 0.0709 - loss: 0.1418
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - huber_metric: 0.0854 - loss: 0.1707 
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - huber_metric: 0.0524 - loss: 0.1048
